# Лабораторная работа 4. Модель однофазной многоканальной замкнутой системы обслуживания

## Вариант 2

##### Выполнил: Борисочкин М. И. РТ5-81Б

In [ ]:
# Генерация начальных условий
Variant <- 2
set.seed(Variant) 
k <- sample(c(10:25), 1)
m <- sample(c(3:6), 1)
t1 <- sample(c(14:20), 1)
t2 <- sample(c(2:5), 1)
View(data.frame(k, m, t1, t2))

k,m,t1,t2
<int>,<int>,<int>,<int>
14,5,19,3


### Теоритическое решение

С помощью формулы (25) из лекции 3 была выведена следующая формула вероятности пребывания в состоянии $S_0$:

$
P_0 = (∑_{i = 0}^m \frac{k! ⋅ y^i}{(k - i)! ⋅ i!} + ∑_{i = 1}^{k - m} \frac{k! ⋅ y^{m + i}}{(k - m - i)! ⋅ m!m^i})^{-1}
$
, где $ y = \fracλμ = \frac{\frac{1}{t_1}}{\frac{1}{t_2}} = \frac{t_2}{t_1}$

In [ ]:
# Вероятность, что вычислительный сервер будет свободен
y <- (1/t1)/(1/t2)

# Левая половина уравнения
i <- c(0:m)
p0left <- c((factorial(k)*y^i)/(factorial(k - i) * factorial(i)))
p0left <-sum(p0left)

# Правая половина уравнения
i <- c(1:(k - m))
p0right <- c((factorial(k)*y^(m + i))/(factorial(k - m - i) * factorial(m) * m^i))
p0right <- sum(p0right)

# Финальное выражение
p0 <- (p0left + p0right)^-1

Теперь расчитаем остальные вероятности, когда программа будет выполнена сразу же, как только она поступила на терминал: $P_1, P_2, P_3, P_4$

$
P_1 = \frac{kλ}{1!⋅μ}P_0 = ky ⋅ P_0;
\\ P_2 = \frac{k(k - 1)λ^2}{2!⋅μ^2}P_0 = \frac{k(k - 1)y^2}{2} ⋅ P_0;
\\ P_3 = \frac{k(k - 1)(k - 2)λ^3}{3!⋅μ^3}P_0 = \frac{k(k - 1)(k -2 )y^3}{6} ⋅ P_0;
\\ P_4 = \frac{k(k - 1)(k - 2)(k - 3)λ^4}{4!⋅μ^4}P_0 = \frac{k(k - 1)(k - 2)(k - 3)y^4}{24} ⋅ P_0.
$

In [ ]:
# Вероятности состояний до образования очереди (кроме p0)
i <- c(1:m)
pWorking <- c((factorial(k)*y^i)/(factorial(k - i) * factorial(i)) * p0)

In [ ]:
# Вероятность того, что программа не будет выполнена сразу же, как только она поступила на терминал
1 - (p0 + sum(pWorking[1:4]))

[1] 0.03462557

In [ ]:
# Вероятности состояний образования очереди
i <- c(1:(k - m))
pQueue <- c((factorial(k)*y^(m + i))/(factorial(k - m - i) * factorial(m) * m^i)*p0)

Для расчёта среднего времени до получения пользователем результатов реализации используем величину $T_{сист}$:

$
T_{сист} = \frac{L_{сист}}{Λ} = \frac{L_{сист}}{(k - L_{сист})λ}, где \ L_{сист} = ∑_{i = 1}^k i⋅ P_i
$

In [ ]:
# Среднее время до получения пользователем результатов реализации
i <- c(1:k)
Lsys <- sum(pWorking*i[1:m]) + sum(pQueue*i[(m + 1) : k])
Tsys <- Lsys/((k - Lsys)*(1/t1))
Tsys

[1] 3.019411

Для расчёта среднего количества программ, ожидающих выполнения на сервере воспользуемся величиной $L_{оч}$:

$
L_{оч} = ∑_{i = 1}^∞ i ⋅ P_{m + i} = ∑_{i = 1}^{k - m} i ⋅ P_{m + i} = ∑_{i = 1}^{9} i ⋅ P_{5 + i}
$

In [ ]:
# Среднее количество программ, ожидающих выполнения на сервере
Lqueue <- sum(i*pQueue)
Lqueue

[1] 0.01234128

### Эксперементальное решение

In [ ]:
# Установка библиотеки
install.packages("simmer", dependencies = TRUE)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in install.packages("simmer", dependencies = TRUE):
“installation of package ‘simmer’ had non-zero exit status”


In [ ]:
# Подключение библиотеки
library("simmer")

In [ ]:
set.seed(Variant)

# Создание траектории программ
m.queue <- trajectory() %>%
  seize("server", amount=1) %>% # Занятие сервера
  timeout(function() rexp(1, 1/t2)) %>% # Обслуживание программы
  release("server", amount=1)# Освобождение сервера

# Создание окружения симуляции
mm.env <- simmer() %>%
  add_resource("server", capacity=m, queue_size=Inf) %>% # Создание ресурса "Сервер"
  add_generator("program", m.queue, function() rexp(1, (k - get_server_count(mm.env, "server") - get_queue_count(mm.env, "server"))/t1)) %>% # Создание потока программ
  run(until=800000)

# Проведение 50 экспериментов
mm.envs <- lapply(1:50, function(i) {mm.env})

In [ ]:
# Вероятность того, что программа не будет выполнена сразу же,
# как только она поступила на терминал
serverData <- get_mon_resources(mm.envs)
sum(serverData$server == m)/length(serverData$server)

[1] 0.05135576

In [ ]:
# Cреднее время до получения пользователем результатов реализации
programsData <- get_mon_arrivals(mm.envs)
mean(programsData$end_time - programsData$start_time)

[1] 3.019924

In [ ]:
# Среднее количество программ, ожидающих выполнения на сервере
mean(serverData$queue)

[1] 0.02291883

## Случай m = 1

In [ ]:
# Начальные условия
m <- 1
lambda <- 1/t1
mu <- 1/t2
y <- lambda/mu

### Теоритическая часть

Для расчёта требуемых значений воспользуемя формулами (5), (8), (9) и (11) из лекции 4

In [ ]:
# Вероятность того, что программа не будет выполнена сразу же, как только она поступила на терминал
i <- (1:k)
p0 <- (sum(factorial(k)*y^i/factorial(k - i)))^-1

1 - p0

[1] 0.996577

In [ ]:
# Среднее время до получения пользователем результатов реализации
Lsys <- k - (1 - p0) * 1/y
Tsys <- Lsys/((k - Lsys)*lambda)
Tsys

[1] 23.14426

In [ ]:
# Среднее количество программ, ожидающих выполнения на сервере
Lqueue <- Lsys - ((k - Lsys)*lambda)/mu
Lqueue

[1] 13.8178

### Эксперементальная часть

In [ ]:
set.seed(Variant)

# Создание траектории программ
m.queue <- trajectory() %>%
  seize("server", amount=1) %>% # Занятие сервера
  timeout(function() rexp(1, mu)) %>% # Обслуживание программы
  release("server", amount=1)# Освобождение сервера

# Создание окружения симуляции
mm.env <- simmer() %>%
  add_resource("server", capacity=1, queue_size=k-1) %>% # Создание ресурса "Сервер"
  add_generator("program", m.queue, function() rexp(1, (k  - get_server_count(mm.env, "server") - get_queue_count(mm.env, "server"))*lambda)) %>% # Создание потока программ
  run(until=800000)

# Проведение 50 экспериментов
mm.envs <- lapply(1:50, function(i) {mm.env})

In [ ]:
# Вероятность того, что программа не будет выполнена сразу же,
# как только она поступила на терминал
serverData <- get_mon_resources(mm.envs)
sum(serverData$server == m)/length(serverData$server)

[1] 0.9999775

In [ ]:
# Cреднее время до получения пользователем результатов реализации
programsData <- get_mon_arrivals(mm.envs)
mean(programsData$end_time - programsData$start_time)

[1] 17.85395

In [ ]:
# Среднее количество программ, ожидающих выполнения на сервере
mean(serverData$queue)

[1] 11.66617